# Analysis of the results of the NNI+DeCoSTAR experiments of July and August 2018
Cedric Chauve, August 31, 2018

In [11]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Overview

This notebook contains an analysis of the results of the experiments consisting into 
* simulating the evolution of genomes (gene trees and gene orders) using <a href="https://www.biorxiv.org/content/early/2018/06/07/339473">Zombi</a>;
* adding noise by performing NNI tree rearrangements, with various levels of noise;
* reconstructing ancestral adjacencies using <a href="https://doi.org/10.1093/gbe/evx069">DeCoSTAR</a>;
* recording statistics about the scores of reconciliation, DeCoSTAR and linearity of ancestral gene orders
in order to assess if the linearity of ancestral gene orders can be used as a diagnostic measure of correctness of the gene trees.

The experiments were run on two datasets of simulated gene trees, one simulated with no HGT and one simulated with HGT, respectively called the noHGT and HGT datasets.

## Data and results

The data and results of the experiments for the noHGT dataset are available in the directory ../../exp/2018_07_15_D10T0L20I10T10_NNI_DECOSTAR. The data and results of the experiments for the noHGT dataset are available in the directory ../../exp/2018_07_16_D10T10L20I10T10_NNI_DECOSTAR. The data and results of the experiments for the noDUP dataset are available in the directory ../../exp/2018_07_25_D0T10L20I10T10_NNI_DECOSTAR.

In a second time we used a modified species tree, differing from the true tree by a single NNI. But only one experiment coild complete due to errors of DeCoSTAR, whose results are available at ../../exp/2018_08_09_D10T0L20I10T10_NNI_DECOSTAR_ST1.

**TO DO**: describe in more detail the experiments.

In each of these directory, a summary file of the results is available in the file results/summary_1.

## Analysis of the results with the noHGT dataset.

In [3]:
# Reading summary file (path to file, F) and creating a dataframe

# Correspondance between dataset names and lambda parameter of the Poisson law used to determine the average number of NNI per gene tree
LAMBDA={'lambda_025':0.25,'lambda_05':0.5,'lambda_1':1,'lambda_2':2,'lambda_3':3,'lambda_5':5,'lambda_7':7,'lambda_10':10,'lambda_20':20,'lambda_30':30,'lambda_50'
:50}

def read_summary_file(F):
    SUMMARY_FILE = open(F,'r').readlines()
    SUMMARY_AUX  = {}
    for l in SUMMARY_FILE:
        if l[0]!="#":
            l1 = l.rstrip().split()
            (name,plambda,rf,gdup,gloss,hgt,rec,again,abreak,deco,linearity) = (l1[0],float(LAMBDA[l1[0]]),float(l1[1]),float(l1[2]),float(l1[3]),float(l1[4]),float(l1[5]),float(l1[6]),float(l1[7]),float(l1[8]),float(l1[9]))
            SUMMARY_AUX[name] = (plambda,rf,gdup,gloss,hgt,rec,again,abreak,deco,linearity)  
    SUMMARY = pd.DataFrame.transpose(pd.DataFrame(SUMMARY_AUX))
    SUMMARY.columns=['Lambda','RF','nb_dup','nb_loss','nb_hgt','rec_score','nb_gain','nb_break','DeCo_score','linearity_score']
    return(SUMMARY)

In [4]:
NOHGT_SUMMARY=read_summary_file('../../exp/2018_07_15_D10T0L20I10T10_NNI_DECOSTAR/results/summary_1')

In [5]:
NOHGT_SUMMARY

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,0.13,183.0,528.0,0.0,894.0,1782.0,294.0,5640.0,592.0
lambda_05,0.50,0.22,273.0,815.0,0.0,1361.0,1961.0,287.0,6170.0,980.0
lambda_1,1.00,0.47,503.0,1537.0,0.0,2543.0,2399.0,279.0,7476.0,1956.0
lambda_2,2.00,0.96,974.0,3027.0,0.0,4975.0,3259.0,249.0,10026.0,3834.0
lambda_3,3.00,1.36,1361.0,4238.0,0.0,6960.0,3932.0,232.0,12028.0,5234.0
lambda_5,5.00,2.16,2097.0,6643.0,0.0,10837.0,5206.0,207.0,15825.0,8014.0
lambda_7,7.00,2.94,2766.0,8896.0,0.0,14428.0,6153.0,177.0,18636.0,10358.0
lambda_10,10.00,3.97,3599.0,11841.0,0.0,19039.0,7467.0,140.0,22541.0,13336.0
lambda_20,20.00,6.88,5923.0,20537.0,0.0,32383.0,10404.0,111.0,31323.0,21152.0
lambda_30,30.00,8.88,7235.0,26320.0,0.0,40790.0,12122.0,88.0,36454.0,26198.0


In [6]:
# Functions to generate the plots

def plot_RF(df_summary,name):
    # RF distance versus lambda
    plt.scatter(df_summary['Lambda'], df_summary['RF'])
    plt.xlabel("Lambda")
    plt.ylabel("RF distance")
    plt.savefig(name+'_RF.png') 
    plt.close()

def plot_scores(df_summary,name,suffix):

    # Plotting the three scores
    plt.scatter(df_summary['RF'], df_summary['rec_score'], c='g', marker='o')
    plt.scatter(df_summary['RF'], df_summary['DeCo_score'], c='b', marker='x')
    plt.scatter(df_summary['RF'], df_summary['linearity_score'], c='r', marker='+')
    plt.xlabel("RF distance")
    plt.ylabel("Scores")
    plt.legend(['reconciliation','DeCoSTAR','linearity'],loc=2)
    plt.savefig(name+'_scores'+suffix+'.png') 
    plt.close()

    # Plotting rec+DeCo scores
    plt.scatter(df_summary['RF'], df_summary['rec_score'], c='g', marker='o')
    plt.scatter(df_summary['RF'], df_summary['DeCo_score'], c='b', marker='x')
    plt.xlabel("RF distance")
    plt.ylabel("Scores")
    plt.legend(['reconciliation','DeCoSTAR'],loc=2)
    plt.savefig(name+'_scores_rec_DeCo'+suffix+'.png') 
    plt.close()

    # Plotting linearity score
    plt.scatter(df_summary['RF'], df_summary['linearity_score'], c='r', marker='+')
    plt.xlabel("RF distance")
    plt.ylabel("Linearity score")
    plt.savefig(name+'_linearity'+suffix+'.png') 
    plt.close()

In [7]:
plot_RF(NOHGT_SUMMARY,'2018_07_15_D10T0L20I10T10_NNI_DECOSTAR')
plot_scores(NOHGT_SUMMARY,'2018_07_15_D10T0L20I10T10_NNI_DECOSTAR','')

<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_RF.png">
<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores.png">
<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png">
<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_linearity.png">

## Analysis of the results with the HGT dataset.

In [8]:
HGT_SUMMARY=read_summary_file('../../exp/2018_07_16_D10T10L20I10T10_NNI_DECOSTAR/results/summary_1')

In [9]:
HGT_SUMMARY

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,0.12,118.0,289.0,192.0,1101.0,1827.0,334.0,5815.0,218.0
lambda_05,0.50,0.25,120.0,388.0,312.0,1564.0,2050.0,330.0,6480.0,260.0
lambda_1,1.00,0.45,117.0,543.0,507.0,2298.0,2443.0,322.0,7651.0,378.0
lambda_2,2.00,0.93,112.0,873.0,952.0,3953.0,3269.0,283.0,10090.0,696.0
lambda_3,3.00,1.38,108.0,1175.0,1342.0,5417.0,3979.0,287.0,12224.0,940.0
lambda_5,5.00,2.18,104.0,1605.0,1951.0,7666.0,5119.0,241.0,15598.0,1486.0
lambda_7,7.00,2.97,93.0,2081.0,2537.0,9878.0,6131.0,240.0,18633.0,1900.0
lambda_10,10.00,3.96,88.0,2577.0,3250.0,12503.0,7396.0,204.0,22392.0,2714.0
lambda_20,20.00,6.65,86.0,3661.0,4805.0,18248.0,10054.0,156.0,30318.0,4554.0
lambda_30,30.00,8.93,103.0,4444.0,5913.0,22389.0,11873.0,121.0,35740.0,6242.0


In [10]:
plot_RF(HGT_SUMMARY,'2018_07_16_D10T10L20I10T10_NNI_DECOSTAR')
plot_scores(HGT_SUMMARY,'2018_07_16_D10T10L20I10T10_NNI_DECOSTAR','')

<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_RF.png">
<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores.png">
<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png">
<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_linearity.png">

## Analysis of the results with the HGT+noDup dataset.

We ran two DeCoSTAR experiments: mone with the usual weighting scheme for reconciliation (experiment 1) and one where both Dup and HGT have weight 2 (experiment 2).

In [11]:
NODUP_SUMMARY_1=read_summary_file('../../exp/2018_07_25_D0T10L20I10T10_NNI_DECOSTAR/results/summary_1')
NODUP_SUMMARY_2=read_summary_file('../../exp/2018_07_25_D0T10L20I10T10_NNI_DECOSTAR/results/summary_2')

In [12]:
NODUP_SUMMARY_1

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,0.12,0.0,279.0,190.0,849.0,1725.0,318.0,5493.0,178.0
lambda_05,0.50,0.23,0.0,369.0,295.0,1254.0,1925.0,305.0,6080.0,252.0
lambda_1,1.00,0.47,0.0,568.0,519.0,2125.0,2384.0,309.0,7461.0,352.0
lambda_2,2.00,0.88,1.0,863.0,903.0,3574.0,3107.0,285.0,9606.0,568.0
lambda_3,3.00,1.35,1.0,1167.0,1297.0,5060.0,3854.0,272.0,11834.0,884.0
lambda_5,5.00,2.11,1.0,1670.0,1901.0,7375.0,4942.0,238.0,15064.0,1280.0
lambda_7,7.00,2.78,2.0,2001.0,2374.0,9127.0,5814.0,238.0,17680.0,1750.0
lambda_10,10.00,3.80,8.0,2496.0,3087.0,11773.0,7028.0,183.0,21267.0,2492.0
lambda_20,20.00,6.57,18.0,3576.0,4717.0,17763.0,9801.0,136.0,29539.0,4524.0
lambda_30,30.00,8.56,45.0,4361.0,5695.0,21536.0,11431.0,101.0,34394.0,5646.0


In [13]:
NODUP_SUMMARY_2

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,0.12,0.0,273.0,193.0,659.0,1731.0,318.0,5511.0,186.0
lambda_05,0.50,0.23,0.0,363.0,298.0,959.0,1934.0,305.0,6107.0,262.0
lambda_1,1.00,0.47,0.0,548.0,528.0,1604.0,2403.0,310.0,7519.0,380.0
lambda_2,2.00,0.88,0.0,848.0,911.0,2670.0,3123.0,285.0,9654.0,602.0
lambda_3,3.00,1.35,0.0,1103.0,1328.0,3759.0,3904.0,270.0,11982.0,964.0
lambda_5,5.00,2.11,1.0,1564.0,1950.0,5466.0,5005.0,231.0,15246.0,1392.0
lambda_7,7.00,2.78,0.0,1826.0,2458.0,6742.0,5923.0,231.0,18000.0,1908.0
lambda_10,10.00,3.80,2.0,2189.0,3232.0,8657.0,7176.0,168.0,21696.0,2728.0
lambda_20,20.00,6.57,1.0,2864.0,5057.0,12980.0,10002.0,109.0,30115.0,4658.0
lambda_30,30.00,8.56,1.0,3248.0,6228.0,15706.0,11719.0,78.0,35235.0,5784.0


In [14]:
plot_RF(NODUP_SUMMARY_1,'2018_07_25_D0T10L20I10T10_NNI_DECOSTAR')
plot_scores(NODUP_SUMMARY_1,'2018_07_25_D0T10L20I10T10_NNI_DECOSTAR','_1')
plot_scores(NODUP_SUMMARY_2,'2018_07_25_D0T10L20I10T10_NNI_DECOSTAR','_2')

<img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_RF.png">

￼<table align="center">
<tr>
    <th>Experiment 1: HGT cost=3, Dup cost=2, Loss cost=1</th>
    <th>Experiment 2: HGT cost=2, Dup cost=2, Loss cost=1</th>
</tr>
￼<tr>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_scores_1.png"></td>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_scores_2.png"></td>
</tr>
<tr>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_scores_rec_DeCo_1.png"></td>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_scores_rec_DeCo_2.png"></td>
</tr>
<tr>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_linearity_1.png"></td>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_linearity_2.png"></td>
</tr>
 </table>

## Analysis of the results with the noHGT + modified species tree dataset.

In [15]:
NOHGT_ST1_SUMMARY=read_summary_file('../../exp/2018_08_09_D10T0L20I10T10_NNI_DECOSTAR_ST1/results/summary_1')
NOHGT_ST1_SUMMARY['RF']=NOHGT_SUMMARY['RF']

In [16]:
NOHGT_ST1_SUMMARY

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,0.13,1151.0,3443.0,0.0,5745.0,1788.0,237.0,5601.0,808.0
lambda_05,0.50,0.22,1216.0,3661.0,0.0,6093.0,1968.0,235.0,6139.0,1178.0
lambda_1,1.00,0.47,1419.0,4318.0,0.0,7156.0,2408.0,224.0,7448.0,2120.0
lambda_2,2.00,0.96,1852.0,5704.0,0.0,9408.0,3261.0,202.0,9985.0,3956.0
lambda_3,3.00,1.36,2179.0,6754.0,0.0,11112.0,3932.0,185.0,11981.0,5342.0
lambda_5,5.00,2.16,2826.0,8909.0,0.0,14561.0,5193.0,166.0,15745.0,8096.0
lambda_7,7.00,2.94,3474.0,11123.0,0.0,18071.0,6131.0,138.0,18531.0,10426.0
lambda_10,10.00,3.97,4137.0,13582.0,0.0,21856.0,7421.0,113.0,22376.0,13290.0
lambda_20,20.00,6.88,6246.0,21610.0,0.0,34102.0,10311.0,98.0,31031.0,20746.0
lambda_30,30.00,8.88,7482.0,27074.0,0.0,42038.0,12026.0,73.0,36151.0,25994.0


In [17]:
plot_scores(NOHGT_ST1_SUMMARY,'2018_08_09_D10T0L20I10T10_NNI_DECOSTAR_ST1','')

We show the results, compared to the same dataset but with the exact species tree.

<table align="center">
<tr>
    <th>Experiment 1: noHGT true species tree</th>
    <th>Experiment 2: noHGT wrong species tree</th>
</tr>
￼<tr>
    <td><img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores.png"></td>
    <td><img src="2018_08_09_D10T0L20I10T10_NNI_DECOSTAR_ST1_scores.png"></td>
</tr>
<tr>
    <td><img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png"></td>
    <td><img src="2018_08_09_D10T0L20I10T10_NNI_DECOSTAR_ST1_scores_rec_DeCo.png"></td>
</tr>
<tr>
    <td><img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_linearity.png"></td>
    <td><img src="2018_08_09_D10T0L20I10T10_NNI_DECOSTAR_ST1_linearity.png"></td>
</tr>
</table>

## Analysis of the results with the HGT + modified species tree dataset.

In [18]:
HGT_ST1_SUMMARY=read_summary_file('../../exp/2018_08_09_D10T10L20I10T10_NNI_DECOSTAR_ST1/results/summary_1')
HGT_ST1_SUMMARY['RF']=HGT_SUMMARY['RF']

In [19]:
HGT_ST1_SUMMARY

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,NaN,115.0,1181.0,1137.0,4822.0,2017.0,307.0,6358.0,390.0
lambda_05,0.50,0.25,118.0,1218.0,1226.0,5132.0,2355.0,295.0,7360.0,554.0
lambda_1,1.00,0.45,114.0,1335.0,1394.0,5745.0,2812.0,275.0,8711.0,730.0
lambda_2,2.00,0.93,107.0,1525.0,1763.0,7028.0,3854.0,229.0,11791.0,1214.0
lambda_3,3.00,1.38,105.0,1759.0,2111.0,8302.0,4661.0,242.0,14225.0,1574.0
lambda_5,5.00,2.18,105.0,2052.0,2587.0,10023.0,5772.0,192.0,17508.0,2080.0
lambda_7,7.00,2.97,96.0,2392.0,3148.0,12028.0,6816.0,196.0,20644.0,2568.0
lambda_10,10.00,3.96,93.0,2829.0,3730.0,14205.0,7996.0,164.0,24152.0,3262.0
lambda_20,20.00,6.65,103.0,3809.0,5119.0,19372.0,10444.0,123.0,31455.0,5052.0
lambda_30,30.00,8.93,112.0,4456.0,6166.0,23178.0,12209.0,98.0,36725.0,6666.0


In [20]:
plot_scores(HGT_ST1_SUMMARY,'2018_08_09_D10T10L20I10T10_NNI_DECOSTAR_ST1','')

We show the results, compared to the same dataset but with the exact species tree.

<table align="center">
<tr>
    <th>Experiment 1: HGT true species tree</th>
    <th>Experiment 2: HGT wrong species tree</th>
</tr>
￼<tr>
    <td><img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores.png"></td>
    <td><img src="2018_08_09_D10T10L20I10T10_NNI_DECOSTAR_ST1_scores.png"></td>
</tr>
<tr>
    <td><img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png"></td>
    <td><img src="2018_08_09_D10T10L20I10T10_NNI_DECOSTAR_ST1_scores_rec_DeCo.png"></td>
</tr>
<tr>
    <td><img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_linearity.png"></td>
    <td><img src="2018_08_09_D10T10L20I10T10_NNI_DECOSTAR_ST1_linearity.png"></td>
</tr>
</table>

## Analysis of the results with the HGT+noDup + modified species tree dataset.

In [21]:
NODUP_ST1_SUMMARY=read_summary_file('../../exp/2018_08_09_D0T10L20I10T10_NNI_DECOSTAR_ST1/results/summary_1')
NODUP_ST1_SUMMARY['RF']=NODUP_SUMMARY_1['RF']

In [22]:
NODUP_ST1_SUMMARY

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,0.12,0.0,1154.0,1094.0,4436.0,1910.0,288.0,6018.0,324.0
lambda_05,0.50,0.23,0.0,1212.0,1187.0,4773.0,2169.0,266.0,6773.0,446.0
lambda_1,1.00,0.47,1.0,1348.0,1384.0,5502.0,2750.0,266.0,8516.0,698.0
lambda_2,2.00,0.88,1.0,1532.0,1695.0,6619.0,3634.0,251.0,11153.0,1052.0
lambda_3,3.00,1.35,1.0,1679.0,2052.0,7837.0,4473.0,217.0,13636.0,1560.0
lambda_5,5.00,2.11,4.0,2087.0,2553.0,9754.0,5600.0,188.0,16988.0,2002.0
lambda_7,7.00,2.78,3.0,2332.0,2987.0,11299.0,6513.0,176.0,19715.0,2538.0
lambda_10,10.00,3.80,10.0,2697.0,3563.0,13406.0,7616.0,141.0,22989.0,3258.0
lambda_20,20.00,6.57,22.0,3600.0,5071.0,18857.0,10222.0,113.0,30779.0,5006.0
lambda_30,30.00,8.56,48.0,4387.0,5917.0,22234.0,11692.0,102.0,35178.0,6174.0


In [23]:
plot_scores(HGT_ST1_SUMMARY,'2018_08_09_D0T10L20I10T10_NNI_DECOSTAR_ST1','')

We show the results, compared to the same dataset but with the exact species tree.

<table align="center">
<tr>
    <th>Experiment 1: HGT true species tree</th>
    <th>Experiment 2: HGT wrong species tree</th>
</tr>
￼<tr>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_scores_1.png"></td>
    <td><img src="2018_08_09_D0T10L20I10T10_NNI_DECOSTAR_ST1_scores.png"></td>
</tr>
<tr>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_scores_rec_DeCo_1.png"></td>
    <td><img src="2018_08_09_D0T10L20I10T10_NNI_DECOSTAR_ST1_scores_rec_DeCo.png"></td>
</tr>
<tr>
    <td><img src="2018_07_25_D0T10L20I10T10_NNI_DECOSTAR_linearity_1.png"></td>
    <td><img src="2018_08_09_D0T10L20I10T10_NNI_DECOSTAR_ST1_linearity.png"></td>
</tr>
</table>